## Description of the problem

In Munich, if someone is looking to open a restaurant, where is the best place to open and what kind of restaurant is missing there? Munich is a city with a lot of different types of restaurants (italian, bavarian, german, indian, asian, etc. ). Alex our persona wants to open a restaurants in Munich, he is experienced in Indian, Asian and Italian food and can not decide which of these restaurants he should open, so he can make more profit. So he wants to analyze the district around the store and find the best possible solution to open his restaurant. 

questions to be answered:
1. what kind of restaurants are the most common ones in each district around his store?
2. how do the districts differ from each other?


## Description of the data


Data from Foursquare will help him to identify the most common restaurant in each area, after clustering the areas suggestions will be made to find out what kind of restaurant is missing or is better to open in that area through Machine learning. As the API for Foursquare in the regular mode is restricted, the decision making will just be based on the distance and types of restaurant around his defined stores.

## Code



### Importing Libraries



In [115]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library


Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


### Define Foursquare Credentials and Version

In [123]:
# The code was removed by Watson Studio for sharing.

In [124]:
address = 'Theresienstrasse, Munich, MU'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


48.150795 11.5654268


### Indian Restaurant

#### Search query and URL for 

In [223]:
search_query = 'Indian'
radius = 3000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()

#### Get relevant part of JSON and transform it into a *pandas* dataframe

In [224]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)

#### Define information of interest and filter dataframe

In [225]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

df_indian =dataframe_filtered.loc[dataframe_filtered['categories'] == 'Indian Restaurant']

### Repeat last step for asian and italian cuisine

In [226]:
### Asian
search_query = 'Asian'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe

# filter the category for each row
dataframe_filtered = dataframe.loc[:, filtered_columns]
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered
df_asian =dataframe_filtered.loc[dataframe_filtered['categories'] == 'Asian Restaurant']

###Italian
search_query = 'Italian'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()

# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe

# filter the category for each row
dataframe_filtered = dataframe.loc[:, filtered_columns]
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered
df_italian =dataframe_filtered.loc[dataframe_filtered['categories'] == 'Italian Restaurant']


### Merge the dataframe of the different restaurants in one dataframe

In [227]:
frames = [df_indian, df_asian, df_italian]
df_total = pd.concat(frames)
df_total

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Indian,Indian Restaurant,NaN,DE,München,Deutschland,1151,"[München, Deutschland]","[{'lat': 48.14313640348064, 'label': 'display', 'lng': 11.575846734917425}]",48.143136,11.575847,NaN,Bayern,50dd37c2e4b021301d34f7dc
1,Indian Mango,Indian Restaurant,Zweibrückenstr. 15,DE,München,Deutschland,2443,"[Zweibrückenstr. 15, 80331 München, Deutschland]","[{'lat': 48.132733841549275, 'label': 'display', 'lng': 11.58412135269005}]",48.132734,11.584121,80331,Bayern,4f6f175ae4b0ad1af604f632
2,Goa Indian Restaurant,Indian Restaurant,Thierschstr. 8,DE,München,Deutschland,2311,"[Thierschstr. 8, 80538 München, Deutschland]","[{'lat': 48.13419460779889, 'label': 'display', 'lng': 11.584111027422885}]",48.134195,11.584111,80538,Bayern,4bb33e3f4019a593e2a737b8
3,Indian Sun,Indian Restaurant,Tumblingerstraße 24,DE,München,Deutschland,2799,"[Tumblingerstraße 24, 80337 München, Deutschland]","[{'lat': 48.126168, 'label': 'display', 'lng': 11.557805}]",48.126168,11.557805,80337,Bayern,5bc9b4b167f62b002c1ee546
6,Lahori Pakistanisch-Indisch Restaurant,Indian Restaurant,Schillerstr.3a/UG,DE,München,Deutschland,1382,"[Schillerstr.3a/UG, 80336 München, Deutschland]","[{'lat': 48.13870130960062, 'label': 'display', 'lng': 11.561165080844797}]",48.138701,11.561165,80336,Bayern,4e2eb692d1646b407b86144d
0,Asian Corner,Asian Restaurant,Seidlstr. 5,DE,München,Deutschland,1098,"[Seidlstr. 5, 80335 München, Deutschland]","[{'lat': 48.14291533764868, 'label': 'display', 'lng': 11.556525676947663}]",48.142915,11.556526,80335,Bayern,4e6b1c14d164c37bf4e07d2e
1,Asian Cuisine,Asian Restaurant,Wittelsbacher Platz 1,DE,München,Deutschland,1140,"[Wittelsbacher Platz 1, 80333 München, Deutschland]","[{'lat': 48.14309, 'label': 'display', 'lng': 11.575538}]",48.143090,11.575538,80333,Bayern,4fb62885e4b0bf7a6c01f2b7
2,Thai-China Asian Quick-Food,Asian Restaurant,Bahnhofplatz 1,DE,München,Deutschland,1221,"[Bahnhofplatz 1 (Schützenstraße), 80335 München, Deutschland]","[{'lat': 48.140095932820316, 'label': 'display', 'lng': 11.561795875186618}]",48.140096,11.561796,80335,Bayern,4be93750ee96c928c4e4febf
4,Sushi Bar and Asian Wok,Asian Restaurant,Paul-Heyse Strasse 2,DE,München,Deutschland,1870,"[Paul-Heyse Strasse 2, München, Deutschland]","[{'lat': 48.13487387504588, 'label': 'display', 'lng': 11.55736058525335}]",48.134874,11.557361,NaN,Bayern,4fe339b0e4b08209955092a6
1,The Italian Shot,Italian Restaurant,Theresienstr. 40,DE,München,Deutschland,776,"[Theresienstr. 40, 80333 München, Deutschland]","[{'lat': 48.14811439351411, 'label': 'display', 'lng': 11.575081465101965}]",48.148114,11.575081,80333,Bayern,5852ed52ac3291412330aa68


### Investigating the restaurant around our store

In [234]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(df_total.lat, df_total.lng, df_total.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

### Import libraries for clusteranalysis

In [235]:
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs 
%matplotlib inline

#### Changing categort to integers

In [236]:
### we want to cluster the restaurants based on the category and distance so instead of category names we need integers
#First, change the type of the column:
df_total.categories = pd.Categorical(df_total.categories)
#Now the data look similar but are stored categorically. To capture the category codes:

df_total['code'] = df_total.categories.cat.codes

filtered_columns = ['code', 'distance'] 
df_tf = df_total.loc[:, filtered_columns] #filtered total
df_tf

,code,distance
0,1,1151
1,1,2443
2,1,2311
3,1,2799
6,1,1382
0,0,1098
1,0,1140
2,0,1221
4,0,1870
1,2,776


### k-means analysis

In [238]:
kclusters = 3

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_tf)

df_total['cluster']=kmeans.labels_
df_total

,name,categories,address,cc,city,country,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id,code,cluster
0,Indian,Indian Restaurant,NaN,DE,München,Deutschland,1151,"[München, Deutschland]","[{'lat': 48.14313640348064, 'label': 'display', 'lng': 11.575846734917425}]",48.143136,11.575847,NaN,Bayern,50dd37c2e4b021301d34f7dc,1,1
1,Indian Mango,Indian Restaurant,Zweibrückenstr. 15,DE,München,Deutschland,2443,"[Zweibrückenstr. 15, 80331 München, Deutschland]","[{'lat': 48.132733841549275, 'label': 'display', 'lng': 11.58412135269005}]",48.132734,11.584121,80331,Bayern,4f6f175ae4b0ad1af604f632,1,2
2,Goa Indian Restaurant,Indian Restaurant,Thierschstr. 8,DE,München,Deutschland,2311,"[Thierschstr. 8, 80538 München, Deutschland]","[{'lat': 48.13419460779889, 'label': 'display', 'lng': 11.584111027422885}]",48.134195,11.584111,80538,Bayern,4bb33e3f4019a593e2a737b8,1,2
3,Indian Sun,Indian Restaurant,Tumblingerstraße 24,DE,München,Deutschland,2799,"[Tumblingerstraße 24, 80337 München, Deutschland]","[{'lat': 48.126168, 'label': 'display', 'lng': 11.557805}]",48.126168,11.557805,80337,Bayern,5bc9b4b167f62b002c1ee546,1,0
6,Lahori Pakistanisch-Indisch Restaurant,Indian Restaurant,Schillerstr.3a/UG,DE,München,Deutschland,1382,"[Schillerstr.3a/UG, 80336 München, Deutschland]","[{'lat': 48.13870130960062, 'label': 'display', 'lng': 11.561165080844797}]",48.138701,11.561165,80336,Bayern,4e2eb692d1646b407b86144d,1,1
0,Asian Corner,Asian Restaurant,Seidlstr. 5,DE,München,Deutschland,1098,"[Seidlstr. 5, 80335 München, Deutschland]","[{'lat': 48.14291533764868, 'label': 'display', 'lng': 11.556525676947663}]",48.142915,11.556526,80335,Bayern,4e6b1c14d164c37bf4e07d2e,0,1
1,Asian Cuisine,Asian Restaurant,Wittelsbacher Platz 1,DE,München,Deutschland,1140,"[Wittelsbacher Platz 1, 80333 München, Deutschland]","[{'lat': 48.14309, 'label': 'display', 'lng': 11.575538}]",48.143090,11.575538,80333,Bayern,4fb62885e4b0bf7a6c01f2b7,0,1
2,Thai-China Asian Quick-Food,Asian Restaurant,Bahnhofplatz 1,DE,München,Deutschland,1221,"[Bahnhofplatz 1 (Schützenstraße), 80335 München, Deutschland]","[{'lat': 48.140095932820316, 'label': 'display', 'lng': 11.561795875186618}]",48.140096,11.561796,80335,Bayern,4be93750ee96c928c4e4febf,0,1
4,Sushi Bar and Asian Wok,Asian Restaurant,Paul-Heyse Strasse 2,DE,München,Deutschland,1870,"[Paul-Heyse Strasse 2, München, Deutschland]","[{'lat': 48.13487387504588, 'label': 'display', 'lng': 11.55736058525335}]",48.134874,11.557361,NaN,Bayern,4fe339b0e4b08209955092a6,0,2
1,The Italian Shot,Italian Restaurant,Theresienstr. 40,DE,München,Deutschland,776,"[Theresienstr. 40, 80333 München, Deutschland]","[{'lat': 48.14811439351411, 'label': 'display', 'lng': 11.575081465101965}]",48.148114,11.575081,80333,Bayern,5852ed52ac3291412330aa68,2,1


In [240]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco
color_df = 'red', 'blue', 'yellow', 'green'

# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='black',
    fill_color='black',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label, cluster in zip(df_total.lat, df_total.lng, df_total.categories, df_total.cluster):
    folium.features.CircleMarker(
        [lat, lng],
        radius=15,
        popup=label,
        fill=True,
        color= color_df[cluster-1],
        fill_color=color_df[cluster-1],
        fill_opacity=0.8
        ).add_to(venues_map)

# display map
venues_map